In [1]:
import ps, ps_utils
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
ps_utils.switch_to_dev()
no_skew_model = 'HybridLGM1FDupireLocalVolatilityModel'
skew_model = 'HybridCheyette1FactorLocalVolatilityModel'
import warnings
warnings.filterwarnings("ignore") 
from ps import message as msg

from ps_utils.env import ps
import numpy as np
from ps_utils import env, market, analytics, pricing
import pandas as pd

from typing import Union, Optional, Dict, List
PsObject = Union[ps.Message, ps.GlobalID]


ps_utils.global_algo_id()

[INFO] 2023-10-03 11:08:22,025 - Your ps_utils version is 23.1.1, it is up to date
[WARNING] 2023-10-03 11:08:23,496 - You do not have xbbg installed, some functionality will not be available
[INFO] 2023-10-03 11:08:24,419 - Nothing to do: current environment is already dev


'8c2e14594fe4595354c9a07103c0d00b7a6d6923'

In [2]:
def prepare_big_request_for_pnl(response, prev_response, algo_id: str = None, reporting_currency: str = None, queries: List[str] = None) -> msg.MessageList:

    if algo_id is None:
        algo_id = env.global_algo_id()

    big_request = ps.new_cm([])
    new_resp = ps.get(ps.put(response))
    new_prev_resp = ps.get(ps.put(prev_response))

    for r in new_resp:
        cpty = r.Request['_Counterparty']
        prev_r = next((x for x in new_prev_resp if x.Request['_Counterparty'] == cpty), None)
        req = ps.new_cm({
            'RequestName': 'Layer2',
            '_Counterparty': cpty,
            'Layer2Algo': algo_id,
            'ReportingCurrency': reporting_currency if reporting_currency else r.Request.ReportingCurrency,
            'Method': 'Revaluation',
            'Queries': queries or r.Request.Queries,
            'Request': r.Request,
            'Response': r,
        })
        if prev_r:
            new_prev_resp.remove(prev_r)
            req['PreviousRequest'] = prev_r.Request
            req['PreviousResponse'] = prev_r
        big_request.append(req)

    for r in new_prev_resp:
        cpty = r.Request['_Counterparty']
        big_request.append(ps.new_cm({
            'RequestName': 'Layer2',
            '_Counterparty': cpty,
            'Layer2Algo': algo_id,
            'ReportingCurrency': reporting_currency if reporting_currency else r.Request.ReportingCurrency,
            'Method': 'Revaluation',
            'Queries': queries or r.Request.Queries,
            'PreviousRequest': r.Request,
            'PreviousResponse': r
        }))

    return ps.new_cm(big_request)


def __calculate_pnl_category(request: msg.MessageList, prev_request: msg.MessageList, results: msg.MessageList, algo_id: str, category: str):
    # Calculating request
    calc_request = ps.compute_raw(algo_id, request).Result
    calc_request = [x for x in calc_request if x.get('StatusText') is not None and x.StatusText == 'done']
    calc_prev_request = ps.compute_raw(algo_id, prev_request).Result
    calc_prev_request = [x for x in calc_prev_request if x.get('StatusText') is not None and x.StatusText == 'done']

    # Matching results
    for res in calc_request:
        cpty = res.Request['_Counterparty']
        result = next((x for x in results if x.Request['_Counterparty'] == cpty))
        prev_res = next((x for x in calc_prev_request if x.Request['_Counterparty'] == cpty), None)
        for query in res.Request.Queries:
            result[query][category] = res.Result[query] - prev_res.Result[query] if prev_res else 0
        if prev_res is not None:
            calc_prev_request.remove(prev_res)
        if category=='Product':
            s2=set()
            try:
                for a in ps.get(res._nonce).Product.NettingSets:
                    for b in a.MarginSets:
                        for el in b.Products:
                            if (el.TypeName=='CompositeProduct'):
                                s2.add(el.Legs[0]['_ContractID'])
                            else:
                                s2.add(el['_ContractID'])
            except:
                pass
            s1=set()
            try:
                for a in ps.get(prev_res._nonce).Product.NettingSets:
                    for b in a.MarginSets:
                        for el in b.Products:
                            s1.add(el['_ContractID'])
            except:
                pass
            result['ExpiredIds1']=ps.new_cm(list(s1.difference(s2)))
            result['NewIds1']=ps.new_cm(list(s2.difference(s1)))
        

    for res in calc_prev_request:
        cpty = res.Request['_Counterparty']
        result = next((x for x in results if x.Request['_Counterparty'] == cpty))
        for query in res.Request.Queries:
            result[query][category] = 0
        if category=='Product':
            s2=set()
            try:
                for a in ps.get(res._nonce).Product.NettingSets:
                    for b in a.MarginSets:
                        for el in b.Products:
                            s2.add(el['_ContractID'])
            except:
                pass
            s1=set()
            try:
                #r=ps.get('de0fc77cbb8b5fd9f8ad3fcfaba3b94469355427')
                for a in ps.get(prev_res._nonce).Product.NettingSets:
                    for b in a.MarginSets:
                        for el in b.Products:
                            s1.add(el['_ContractID'])
            except:
                pass
            result['ExpiredIds2']=ps.new_cm(list(s1.difference(s2)))
            result['NewIds2']=ps.new_cm(list(s2.difference(s1)))
       
            
def __update_marketDependencies(res_market_dependencies: msg.MessageList, results: msg.MessageList, all_market_dependencies: dict, excluded_dependencies: dict):

    for res_dep in res_market_dependencies:
        cpty = res_dep.Request['_Counterparty']
        result = next(filter(lambda x: x.Request['_Counterparty'] == cpty, results))

        for Type in res_dep.Result.MarketDependencies:
            if Type == 'Fixings':
                continue

            for Identifier in res_dep.Result.MarketDependencies[Type]:
                if excluded_dependencies.get(Type) and Identifier in excluded_dependencies.get(Type):
                    continue

                md_to = result.Request.Request.Model.MarketDataSet[Type][Identifier]
                md_from = result.Request.PreviousRequest.Model.MarketDataSet[Type][Identifier]

                if isinstance(md_to, float):
                    if md_to != md_from:
                        result.MarketDependencies.setdefault(Type, [])
                        result.MarketDependencies[Type].append(Identifier)

                        all_market_dependencies.setdefault(Type, [])
                        if Identifier not in all_market_dependencies[Type]:
                            all_market_dependencies[Type].append(Identifier)
                elif md_to._id != md_from._id:
                    result.MarketDependencies.setdefault(Type, [])
                    result.MarketDependencies[Type].append(Identifier)

                    all_market_dependencies.setdefault(Type, [])
                    if Identifier not in all_market_dependencies[Type]:
                        all_market_dependencies[Type].append(Identifier)


def run_generic_revaluation_based_metric_change_explain_batch(big_request: msg.MessageList,
                                                              algo_id: str = None,
                                                              same_mds: bool = False,
                                                              threshold: float = -1,
                                                              optimize_credit_curves: bool = False) -> PsObject:
    """Generates revaluation based explanation report for the metrics requested.
    PS! PnL Explain is calculated from the same Base (compared to other methods)
    PPS! Make sure, input data is in correct format
    PPPS! One request per one cpty in big request!

    Args:
        big_request (MessageList): A list of by-counterparty collected requests on two compared dates
        algo_id (str): ID of the algo to use
        same_mds (bool): True if within date and within prev_date the same mds is assigned (improves performance)
        threshold: if Total change is less than threshold, then PnL Explain for given cpty is not calculated

    Returns:
        PsObject: revaluation based PnL explain summary
    """

    if algo_id is None:
        algo_id = env.global_algo_id()

    results = ps.new_cm([])

    env.logger.info('Calculating Total/Expired/New and removing failed cpties')

    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    for req in big_request:
        if req.get('Request'):
            req.Request.Queries = req.Queries
            req.Request.ReportingCurrency = req.ReportingCurrency
            request.append(req.Request)
        if req.get('PreviousRequest'):
            req.PreviousRequest.Queries = req.Queries
            req.PreviousRequest.ReportingCurrency = req.ReportingCurrency
            prev_request.append(req.PreviousRequest)

    ps.put(request)
    ps.put(prev_request)

    env.logger.info(f'Requests collected. Request={request._id}. PrevRequest={prev_request._id}')
    response = ps.compute_raw(algo_id, request)
    prev_response = ps.compute_raw(algo_id, prev_request)

    for req in big_request:
        cpty = req['_Counterparty']
        result = ps.new_cm({
            '_Counterparty': cpty,
            'Request': req,
        })

        if not req.get('Request'):
            # Expired cpty
            prev_res = next(filter(lambda x: x.Request['_Counterparty'] == cpty, prev_response.Result), None)
            if prev_res.get('StatusText') and prev_res.get('StatusText') == 'done':
                result.update({
                    query: ps.new_cm({
                        'Total': -req.PreviousResponse.Result[query],
                        'ExpiredCounterparty': -req.PreviousResponse.Result[query]
                    }) for query in req.Queries
                })
            else:
                result.update({
                    query: ps.new_cm({'Total': 0}) for query in req.Queries
                })
            result['_NeedsToBeCalculated'] = False
        elif not req.get('PreviousRequest'):
            # New cpty
            res = next(filter(lambda x: x.Request['_Counterparty'] == cpty, response.Result), None)
            if res.get('StatusText') and res.get('StatusText') == 'done':
                result.update({
                    query: ps.new_cm({
                        'Total': req.Response.Result[query],
                        'NewCounterparty': req.Response.Result[query]
                    }) for query in req.Queries
                })
            else:
                result.update({
                    query: ps.new_cm({'Total': 0}) for query in req.Queries
                })
            result['_NeedsToBeCalculated'] = False
        else:
            res = next(filter(lambda x: x.Request['_Counterparty'] == cpty, response.Result), None)
            prev_res = next(filter(lambda x: x.Request['_Counterparty'] == cpty, prev_response.Result), None)

            if res.get('StatusText') and res.get('StatusText') == 'done' and prev_res.get('StatusText') and prev_res.get('StatusText') == 'done':
                result.update({
                    query: ps.new_cm({
                        'Total': req.Response.Result[query] - req.PreviousResponse.Result[query]
                    }) for query in req.Queries
                })
                result['_NeedsToBeCalculated'] = any([abs(result[query]['Total']) > threshold for query in req.Queries])
            elif res.get('StatusText') and res.get('StatusText') == 'done':
                result.update({
                    query: ps.new_cm({
                        'Total': req.Response.Result[query],
                        'ErrorPrevious': req.Response.Result[query]
                    }) for query in req.Queries
                })
                result['_NeedsToBeCalculated'] = False
            elif prev_res.get('StatusText') and prev_res.get('StatusText') == 'done':
                result.update({
                    query: ps.new_cm({
                        'Total': -req.PreviousResponse.Result[query],
                        'ErrorCurrent': -req.PreviousResponse.Result[query]
                    }) for query in req.Queries
                })
                result['_NeedsToBeCalculated'] = False
            else:
                result.update({
                    query: ps.new_cm({
                        'Total': 0,
                        'ErrorBoth': 0
                    }) for query in req.Queries
                })
                result['_NeedsToBeCalculated'] = False

        results.append(result)

    env.logger.info('Total calculated')

    env.logger.info('Doing some preparations')

    env.logger.info('Converting market data to canonical format')
    # Converting market data to canonical format
    if same_mds:
        mds_to = market.to_canonical_format(next(filter(lambda x: x['_NeedsToBeCalculated'], results), None).Request.Request.Model.MarketDataSet, algo_id)
        mds_from = market.to_canonical_format(next(filter(lambda x: x['_NeedsToBeCalculated'], results), None).Request.PreviousRequest.Model.MarketDataSet, algo_id)
        for request in big_request:
            if request.get('Request', None) is not None:
                request.Request.Model.MarketDataSet = mds_to
            if request.get('PreviousRequest', None) is not None:
                request.PreviousRequest.Model.MarketDataSet = mds_from
    else:
        for result in results:
            result.Request.Request.Model.MarketDataSet = market.to_canonical_format(result.Request.Request.Model.MarketDataSet, result.Request.Request.StaticDataSet, algo_id)
            result.Request.PreviousRequest.Model.MarketDataSet = market.to_canonical_format(result.Request.PreviousRequest.Model.MarketDataSet, result.Request.PreviousRequest.StaticDataSet, algo_id)
    env.logger.info('Converting done')

    env.logger.info('Collecting dependencies')

    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    for res in results:
        if res['_NeedsToBeCalculated']:
            res['MarketDependencies'] = {}
            req = ps.get(ps.put(res.Request.Request))
            req.Queries = ['MarketDependencies']
            request.append(req)
            req = ps.get(ps.put(res.Request.PreviousRequest))
            req.Queries = ['MarketDependencies']
            prev_request.append(req)

    ps.put(request)
    ps.put(prev_request)
    env.logger.info(f'Requests collected. Request={request._id}. PrevRequest={prev_request._id}')

    res_market_dependencies = ps.compute_raw(algo_id, request).Result
    res_prev_market_dependencies = ps.compute_raw(algo_id, prev_request).Result

    all_market_dependencies = {}
    excluded_dependencies = {}
    __update_marketDependencies(res_market_dependencies, results, all_market_dependencies, excluded_dependencies)
    __update_marketDependencies(res_prev_market_dependencies, results, all_market_dependencies, excluded_dependencies)

    del res_market_dependencies
    del res_prev_market_dependencies

    env.logger.info('Market dependencies collected')

    env.logger.info('Preparations done')

    # Calculating StaticDataSet change effect
    env.logger.info('Calculating StaticDataSet change effect')
    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    needs_to_be_calculated = False
    for res in results:
        if res['_NeedsToBeCalculated']:
            req = res.Request
            if res['_NeedsToBeCalculated'] and req.Request.StaticDataSet._id != req.PreviousRequest.StaticDataSet._id:
                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                new_req.StaticDataSet = req.Request.StaticDataSet
                request.append(new_req)
                needs_to_be_calculated = True
    if needs_to_be_calculated:
        __calculate_pnl_category(request, prev_request, results, algo_id, 'StaticDataSet')
        env.logger.info('StaticDataSet change calculated')
    else:
        env.logger.info('StaticDataSet change not needed')

    # Calculating Product change effect
    env.logger.info('Calculating Product change effect')
    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    needs_to_be_calculated = False
    for res in results:
        if res['_NeedsToBeCalculated']:
            req = res.Request
            ns_from = ps.put(ps.new_cm(req.PreviousRequest.Product.NettingSets))
            ns_to = ps.put(ps.new_cm(req.Request.Product.NettingSets))
            if res['_NeedsToBeCalculated'] and ns_from._id != ns_to._id:
                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                new_req.Product.NettingSets = req.Request.Product.NettingSets
                request.append(new_req)
                needs_to_be_calculated = True
                continue
    if needs_to_be_calculated:
        __calculate_pnl_category(request, prev_request, results, algo_id, 'Product')
        env.logger.info('Product change calculated')
    else:
        env.logger.info('Product change not needed')

    # Check if cpties were switched from credit curve to internal rating or vice versa
    env.logger.info('Credit model change')
    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    needs_to_be_calculated = False
    for res in results:
        if res['_NeedsToBeCalculated']:
            req = res.Request
            if 'CounterpartyCreditRating' in req.Request.Product and 'CounterpartyCreditRating' not in req.PreviousRequest.Product:
                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                del new_req.Product['CounterpartyCreditCurveIdentifier']
                new_req.Product['CounterpartyCreditRating'] = req.Request.Product.CounterpartyCreditRating
                request.append(new_req)
                needs_to_be_calculated = True
            if 'CounterpartyCreditRating' not in req.Request.Product and 'CounterpartyCreditRating' in req.PreviousRequest.Product:
                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                del new_req.Product['CounterpartyCreditRating']
                new_req.Product['CounterpartyCreditCurveIdentifier'] = req.Request.Product.OwnCreditCurveIdentifier
                request.append(new_req)
                needs_to_be_calculated = True
    if needs_to_be_calculated:
        ps.put(request)
        ps.put(prev_request)
        env.logger.info(f'Requests collected. Request={request._id}. PrevRequest={prev_request._id}')
        __calculate_pnl_category(request, prev_request, results, algo_id, 'CreditModel')
        env.logger.info('Credit model change calculated')
    else:
        env.logger.info('Credit model change not needed')

    # Calculating Rating change effect
    env.logger.info('Calculating Rating change effect')
    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    needs_to_be_calculated = False
    for res in results:
        if res['_NeedsToBeCalculated']:
            req = res.Request
            # If a cpty is calculated on its own curve, than we miss it
            if 'CounterpartyCreditRating' not in req.Request.Product or 'CounterpartyCreditRating' not in req.PreviousRequest.Product:
                continue
            if req.Request.Product.CounterpartyCreditRating != req.PreviousRequest.Product.CounterpartyCreditRating:
                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                new_req.Product.CounterpartyCreditRating = req.Request.Product.CounterpartyCreditRating
                request.append(new_req)
                needs_to_be_calculated = True
    if needs_to_be_calculated:
        __calculate_pnl_category(request, prev_request, results, algo_id, 'Rating')
        env.logger.info('Rating change calculated')
    else:
        env.logger.info('Rating change not needed')

    # Calculating LGD change effect
    env.logger.info('Calculating LGD change effect')
    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    needs_to_be_calculated = False
    for res in results:
        if res['_NeedsToBeCalculated']:
            req = res.Request
            if req.Request.Product.CounterpartyLossGivenDefault != req.PreviousRequest.Product.CounterpartyLossGivenDefault:
                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                new_req.Product.CounterpartyLossGivenDefault = req.Request.Product.CounterpartyLossGivenDefault
                request.append(new_req)
                needs_to_be_calculated = True
    if needs_to_be_calculated:
        __calculate_pnl_category(request, prev_request, results, algo_id, 'LGD')
        env.logger.info('LGD change calculated')
    else:
        env.logger.info('LGD change not needed')
        
    # Calculating NettingSetLGD  change effect
    env.logger.info('Calculating NettingSetLGD change effect')
    request = ps.new_cm([])
    prev_request = ps.new_cm([])
    needs_to_be_calculated = False
    for res in results:
        if res['_NeedsToBeCalculated']:
            req = res.Request
            
            req_dict=dict()
            prevreq_dict=dict()
            
            for el in req.Request.Product.NettingSets:
                if 'NettingSetLossGivenDefault' in el:
                    req_dict[el["_AgreementID"]]=el.NettingSetLossGivenDefault
            for el in req.PreviousRequest.Product.NettingSets:
                if 'NettingSetLossGivenDefault' in el:
                    prevreq_dict[el["_AgreementID"]]=el.NettingSetLossGivenDefault
            
            if (req_dict!=prevreq_dict):
                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                
                for el in new_req.Product.NettingSets:
                    if 'NettingSetLossGivenDefault' in el:
                        el.NettingSetLossGivenDefault=req_dict[el["_AgreementID"]]
                request.append(new_req)
                needs_to_be_calculated = True
    if needs_to_be_calculated:
        __calculate_pnl_category(request, prev_request, results, algo_id, 'NettingSetLGD')
        env.logger.info('NettingSet LGD change calculated')
    else:
        env.logger.info('NettingSet LGD change not needed')

   # Model change
    env.logger.info('Model change effect calculation')
    for model_key in ['NumeraireCurrency', 'TypeName']:

        # First lets check model name and numeraire ccy change
        env.logger.info(model_key + ' change effect calculation')
        request = ps.new_cm([])
        prev_request = ps.new_cm([])
        needs_to_be_calculated = False
        for res in results:
            if res['_NeedsToBeCalculated']:
                req = res.Request
                if req.Request.Model[model_key] != req.PreviousRequest.Model[model_key]:
                    prev_request.append(req.PreviousRequest)
                    new_req = ps.get(ps.put(req.PreviousRequest))
                    new_req.Model[model_key] = req.Request.Model[model_key]
                    request.append(new_req)
                    needs_to_be_calculated = True
        if needs_to_be_calculated:
            __calculate_pnl_category(request, prev_request, result, algo_id, 'Model.' + model_key)
            env.logger.info(model_key + ' change effect done')
        else:
            env.logger.info(model_key + ' change not needed')

    # Theta
    env.logger.info('Theta effect calculation')
    request = []
    prev_request = ps.new_cm([])
    needs_to_be_calculated = False

    if same_mds:
        fixings_from = ps.new_cm(mds_from.Fixings)
        ps.put(fixings_from)
        fixings_to = ps.new_cm(mds_to.Fixings)
        ps.put(fixings_to)

        if mds_to.AsOfDate != mds_from.AsOfDate or fixings_to._id != fixings_from._id:
            request = ps.new_cm([x.Request.PreviousRequest for x in results if x['_NeedsToBeCalculated']])
            request = ps.get(ps.put(request))
            for req in request:
                req.Model.MarketDataSet.AsOfDate = mds_to.AsOfDate
                req.Model.MarketDataSet.Fixings = mds_to.Fixings
            prev_request = ps.new_cm([x.Request.PreviousRequest for x in results if x['_NeedsToBeCalculated']])
            ps.put(prev_request)
            needs_to_be_calculated = True
    else:
        for res in results:
            if res['_NeedsToBeCalculated']:
                req = res.Request

                fixings_from = ps.new_cm(req.PreviousRequest.Model.MarketDataSet.Fixings)
                fixings_to = ps.new_cm(req.Request.Model.MarketDataSet.Fixings)
                ps.put(fixings_from)
                ps.put(fixings_to)

                if req.Request.Model.MarketDataSet.AsOfDate != req.PreviousRequest.Model.MarketDataSet.AsOfDate\
                                                    or fixings_from._id != fixings_to._id:
                    prev_request.append(req.PreviousRequest)
                    new_req = ps.get(ps.put(req.PreviousRequest))
                    new_req.Model.MarketDataSet.AsOfDate = req.Request.Model.MarketDataSet.AsOfDate
                    request.append(new_req)
                    needs_to_be_calculated = True
        if needs_to_be_calculated:
            request = ps.new_cm(request)
            ps.put(request)
            prev_request = ps.new_cm(prev_request)
            ps.put(prev_request)
    if needs_to_be_calculated:
        env.logger.info(f'Requests collected. Request={request._id}. PrevRequest={prev_request._id}')
        __calculate_pnl_category(ps.new_cm(request), ps.new_cm(prev_request), results, algo_id, 'Model.MarketDataSet.AsOfDate')
        env.logger.info('Theta effect done')
    else:
        env.logger.info('Theta not needed')

    # Credit curves
    # Done separately for optimization purposes
    if optimize_credit_curves and all_market_dependencies.get('CreditCurves'):
        env.logger.info('Processing CreditCurves')
        request = []
        prev_request = []
        needs_to_be_calculated = False

        env.logger.info('Collecting requests')
        if same_mds:
            md_to = next(filter(lambda x: x.Request.get('Request') is not None, results)).Request.Request.Model.MarketDataSet
            md_from = next(filter(lambda x: x.Request.get('PreviousRequest') is not None, results)).Request.PreviousRequest.Model.MarketDataSet

            # for curve in all_market_dependencies['CreditCurves']:
            #     if md_to['CreditCurves'][curve]._id != md_from['CreditCurves'][curve]._id:
            #         needs_to_be_calculated = True
            #         break

            new_md_to = ps.get(ps.put(md_from))
            for curve in all_market_dependencies['CreditCurves']:
                new_md_to.CreditCurves[curve] = md_to.CreditCurves[curve]

            for res in results:
                req = res.Request
                if res['_NeedsToBeCalculated'] and 'CreditCurves' in res.MarketDependencies:
                    prev_request.append(req.PreviousRequest)
                    new_req = ps.get(ps.put(req.PreviousRequest))
                    new_req.Model.MarketDataSet = new_md_to
                    request.append(new_req)
                    needs_to_be_calculated = True
        else:
            for res in results:
                req = res.Request
                if res['_NeedsToBeCalculated'] and 'CresitCurves' in res.MarketDependencies:
                    md_to = req.Request.Model.MarketDataSet.CreditCurves
                    md_from = req.PreviousRequest.Model.MarketDataSet.CreditCurves

                    if md_to._id == md_from._id:
                        continue

                prev_request.append(req.PreviousRequest)
                new_req = ps.get(ps.put(req.PreviousRequest))
                new_req.Model.MarketDataSet.CreditCurves = md_to
                request.append(new_req)
                needs_to_be_calculated = True

        if needs_to_be_calculated:
            request = ps.new_cm(request)
            ps.put(request)
            prev_request = ps.new_cm(prev_request)
            ps.put(prev_request)
            env.logger.info(f'Requests collected. Request={request._id}. PrevRequest={prev_request._id}')

            # Calculating and removing unsuccessful results
            calc_request = ps.compute_raw(algo_id, request).Result
            calc_request = [x for x in calc_request if x.get('StatusText') is not None and x.StatusText == 'done']
            calc_prev_request = ps.compute_raw(algo_id, prev_request).Result
            calc_prev_request = [x for x in calc_prev_request if x.get('StatusText') is not None and x.StatusText == 'done']

            # Matching results
            for res in calc_request:
                cpty = res.Request['_Counterparty']
                result = next((x for x in results if x.Request['_Counterparty'] == cpty))
                prev_res = next((x for x in calc_prev_request if x.Request['_Counterparty'] == cpty), None)

                cva_change = res.Result['CVA'] - prev_res.Result['CVA'] if prev_res else None
                cva_curve = res.Request.Product.get('CounterpartyCreditCurveIdentifier') or \
                            res.Request.StaticDataSet.CreditRatingToSpreadMapping[str(res.Request.Product.CounterpartyCreditRating)].BaseCurveIdentifier
                dva_change = res.Result['DVA'] - prev_res.Result['DVA'] if prev_res else None
                dva_curve = res.Request.Product.get('OwnCreditCurveIdentifier')

                if 'ProductValue' in res.Request.Queries:
                    result['ProductValue'][f'Model.MarketDataSet.CreditCurves.{cva_curve}'] = 0
                    result['ProductValue'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] = 0
                if 'CVA' in res.Request.Queries:
                    result['CVA'][f'Model.MarketDataSet.CreditCurves.{cva_curve}'] = cva_change
                    result['CVA'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] = 0
                if 'DVA' in res.Request.Queries:
                    result['DVA'][f'Model.MarketDataSet.CreditCurves.{cva_curve}'] = 0
                    result['DVA'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] = dva_change
                if 'BCVA' in res.Request.Queries:
                    result['BCVA'][f'Model.MarketDataSet.CreditCurves.{cva_curve}'] = cva_change
                    if f'Model.MarketDataSet.CreditCurves.{dva_curve}' not in result['BCVA']:
                        result['BCVA'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] = 0
                    result['BCVA'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] += dva_change

                if prev_res:
                    calc_prev_request.remove(prev_res)

            for res in calc_prev_request:
                cpty = res.Request['_Counterparty']
                result = next((x for x in results if x.Request['_Counterparty'] == cpty))

                cva_curve = res.Request.Product.get('CounterpartyCreditCurveIdentifier') or \
                            res.Request.StaticDataSet.CreditRatingToSpreadMapping[str(res.Request.Product.CounterpartyCreditRating)].BaseCurveIdentifier
                dva_curve = res.Request.Product.get('OwnCreditCurveIdentifier')

                result['CVA'][f'Model.MarketDataSet.CreditCurves.{cva_curve}'] = 0
                result['CVA'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] = 0
                result['DVA'][f'Model.MarketDataSet.CreditCurves.{cva_curve}'] = 0
                result['DVA'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] = 0
                result['BCVA'][f'Model.MarketDataSet.CreditCurves.{cva_curve}'] = 0
                result['BCVA'][f'Model.MarketDataSet.CreditCurves.{dva_curve}'] = 0


        all_market_dependencies.pop('CreditCurves')

    # Other market data
    for Type in all_market_dependencies:
        for Identifier in all_market_dependencies[Type]:
            env.logger.info(f'Processing {Type}.{Identifier}')
            request = []
            prev_request = []
            needs_to_be_calculated = False

            env.logger.info('Collecting requests')
            if same_mds:

                md_to = next(filter(lambda x: x.Request.get('Request') is not None, results)).Request.Request.Model.MarketDataSet
                md_from = next(filter(lambda x: x.Request.get('PreviousRequest') is not None, results)).Request.PreviousRequest.Model.MarketDataSet

                if isinstance(md_to[Type][Identifier], float):
                    if md_to[Type][Identifier] == md_from[Type][Identifier]:
                        continue
                elif md_to[Type][Identifier]._id == md_from[Type][Identifier]._id:
                    continue

                new_md_to = ps.get(ps.put(md_from))
                new_md_to[Type][Identifier] = md_to[Type][Identifier]

                for res in results:
                    req = res.Request
                    if res['_NeedsToBeCalculated'] and Type in res.MarketDependencies \
                            and Identifier in res.MarketDependencies[Type]:

                        prev_request.append(req.PreviousRequest)
                        new_req = ps.get(ps.put(req.PreviousRequest))
                        new_req.Model.MarketDataSet = new_md_to
                        request.append(new_req)
                        needs_to_be_calculated = True
            else:
                for res in results:
                    req = res.Request
                    if res['_NeedsToBeCalculated'] and Type in res.MarketDependencies \
                            and Identifier in res.MarketDependencies[Type]:

                        md_to = req.Request.Model.MarketDataSet[Type][Identifier]
                        md_from = req.PreviousRequest.Model.MarketDataSet[Type][Identifier]

                        if isinstance(md_to, float):
                            if md_to == md_from:
                                continue
                        elif md_to._id == md_from._id:
                            continue

                        prev_request.append(req.PreviousRequest)
                        new_req = ps.get(ps.put(req.PreviousRequest))
                        new_req.Model.MarketDataSet[Type][Identifier] = req.Request.Model.MarketDataSet[Type][Identifier]
                        request.append(new_req)
                        needs_to_be_calculated = True

            if needs_to_be_calculated:
                request = ps.new_cm(request)
                ps.put(request)
                prev_request = ps.new_cm(prev_request)
                ps.put(prev_request)
                env.logger.info(f'Requests collected. Request={request._id}. PrevRequest={prev_request._id}')
                __calculate_pnl_category(request, prev_request, results, algo_id, 'Model.MarketDataSet.' + Type + '.' + Identifier)
                env.logger.info(f'Processing {Type}.{Identifier} done')
            else:
                env.logger.info(f'Processing {Type}.{Identifier} not needed')

    env.logger.info('Removing unnecessary info from result and adding Unexplained')
    for res in results:
        res.pop('MarketDependencies', None)
        res.pop('_NeedsToBeCalculated', None)
        #res.Request.pop('Request', None)
        #res.Request.pop('PreviousRequest', None)
        #res.Request.pop('Response', None)
        #res.Request.pop('PreviousResponse', None)
        if 'Response' in res.Request:
            res.Request.Response=res.Request.Response._id
        if 'PreviousResponse' in res.Request:
            res.Request.PreviousResponse=res.Request.PreviousResponse._id
        for query in res.Request.Queries:
            res[query]['Unexplained'] = 2*res[query]['Total'] - sum(res[query].values())

    env.logger.info('Unnecessary info deleted. Job is done')
    return ps.new_cm(results)



In [3]:
def get_df(response_hashid, isPrev):
    r = ps.get(response_hashid)
    mas_cpty=[]
    mas_resp_hash=[]
    mas_bcva=[]
    for i,el in enumerate(r.Result):
        mas_cpty.append(ps.get(el._nonce)['_Counterparty'])
        mas_resp_hash.append(el._id)
        try:
            mas_bcva.append(el.Result['BCVA'])
        except:
            mas_bcva.append(np.nan)
    df=pd.DataFrame({'Cpty':mas_cpty, 'Resp_Hash_tek':mas_resp_hash,'BCVA_tek':mas_bcva})
    if isPrev:
        df.rename(columns={'Resp_Hash_tek':'Resp_Hash_prev','BCVA_tek':'BCVA_prev'},inplace=True)
    return df

algo_id = '1879d93d4ae4cfddc51b58f0218426de4e6f5da8'
resp_prev=ps.get('a58a9aae38b58af3cb4e1ef2bcf41995efc38209') 
resp_tek=ps.get('0e171ac903da5523603559178a3bd2766ff80b64') 

dtprev = resp_prev['Result'][0]['Request']['Model']['MarketDataSet']['AsOfDate'].date().strftime("%Y-%m-%d")
dttek = resp_tek['Result'][0]['Request']['Model']['MarketDataSet']['AsOfDate'].date().strftime("%Y-%m-%d")

df_prev=get_df(resp_prev, isPrev=True)
df_tek=get_df(resp_tek, isPrev=False)
df=df_prev.merge(df_tek, on='Cpty', how='outer')

newoldcpties=pd.read_excel('OldNewCpties.xlsx')
newoldcpties=newoldcpties[newoldcpties['Metric']=='BCVA']
df=df.merge(newoldcpties,on='Cpty',how='left')

print("PrevBCVA=",df_prev['BCVA_prev'].sum())
print("TekBCVA=",df_tek['BCVA_tek'].sum())
print("diff=",df_tek['BCVA_tek'].sum()-df_prev['BCVA_prev'].sum())

#mask=pd.read_excel('tmp_Names30Cpties.xlsx')['Cpty'].values
#mask=list(map(lambda x: x in mask, df['Cpty']))
#dfmasked=df[mask]
#dfmasked['BCVA_diff']=dfmasked['BCVA_tek']-dfmasked['BCVA_prev']
#dfmasked.head(5)

mask1na=df['BCVA_prev'].isna()
mask2na=df['BCVA_tek'].isna()

# print("New cpties:",df[mask1na]['BCVA_tek'].sum(), df[mask1na]['Cpty'].unique())
# print("Expired cpties:",df[mask2na]['BCVA_prev'].sum(),df[mask2na]['Cpty'].unique())

new_cpties=df[mask1na]
expired_cpties=df[mask2na]
print("New cpties old book:",new_cpties[new_cpties['Old/New']=='Old']['BCVA_tek'].sum(), new_cpties[new_cpties['Old/New']=='Old']['Cpty'].unique())
print("Expired cpties old book:",expired_cpties[expired_cpties['Old/New']=='Old']['BCVA_tek'].sum(), expired_cpties[expired_cpties['Old/New']=='Old']['Cpty'].unique())
print("New cpties new book:",new_cpties[new_cpties['Old/New']=='New']['BCVA_tek'].sum(), new_cpties[new_cpties['Old/New']=='New']['Cpty'].unique())
print("Expired cpties new book:",expired_cpties[expired_cpties['Old/New']=='New']['BCVA_tek'].sum(), expired_cpties[expired_cpties['Old/New']=='New']['Cpty'].unique())


maskna=~(mask1na^mask2na)
dfmasked=df
#dfmasked=df[maskna]
#dfmasked.reset_index()
dfmasked['abs_old_BCVA']=np.abs(dfmasked['BCVA_prev'])
dfmasked['diff_BCVA']=dfmasked['BCVA_tek']-dfmasked['BCVA_prev']
dfmasked['abs_diff_BCVA']=np.abs(dfmasked['diff_BCVA'])
#dfmasked=dfmasked[dfmasked['abs_old_BCVA']>0].reset_index(drop=True)

# newoldcpties=pd.read_excel('OldNewCpties.xlsx')
# newoldcpties=newoldcpties[newoldcpties['Metric']=='BCVA']
# dfmasked=dfmasked.merge(newoldcpties,on='Cpty',how='left')
dfmasked_old=dfmasked[dfmasked['Old/New']=='Old'].reset_index(drop=True)
dfmasked_new=dfmasked[dfmasked['Old/New']=='New'].reset_index(drop=True)
#print("AmtCpties to run pnl explain:",len(dfmasked))
print("Total diff masked=",dfmasked['diff_BCVA'].sum())
print("Old book diff masked=",dfmasked_old['diff_BCVA'].sum())
print("New book diff masked=",dfmasked_new['diff_BCVA'].sum())
dfmasked.sort_values(by='abs_diff_BCVA', ascending=False).head()

PrevBCVA= -16366983.473814739
TekBCVA= -15865958.189086128
diff= 501025.2847286109
New cpties old book: 0.0 ['KVAAT']
Expired cpties old book: 0.0 []
New cpties new book: -6396.359178057695 ['ALMKR' 'CHINK']
Expired cpties new book: 0.0 []
Total diff masked= 506351.51694514963
Old book diff masked= 237796.83800319198
New book diff masked= 268554.67894195777


,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
423,FRTGS,db022b9079f739e6d815697f1243aa7faab01965,"-3,820,497.44",65fe6183a4c5fa1cac6758772273b15cfebd1f3c,"-3,696,640.01",Old,BCVA,"3,820,497.44","123,857.43","123,857.43"
1230,SRBZA,71994a60cb9fd3a939a3f4cc42569861586199c9,-148.26,376f1f2d28ed0677ba705440874a0c1b091a0d38,"-79,334.06",Old,BCVA,148.26,"-79,185.79","79,185.79"
163,MBKRK,5a01b4e8eee77d8ae7ec9ce16f464807c6ee03d6,"-545,714.24",0a31b2061e18ff9d448972ec6acde304ca7209f8,"-477,994.22",New,BCVA,"545,714.24","67,720.03","67,720.03"
914,ROSNB,ef9eb7a2cef9b5bcac7ba80edb9291e109cb9b0a,"567,466.58",5c0ac7be8750d2015290e89dfa169d2b321b5c28,"635,022.24",New,BCVA,"567,466.58","67,555.67","67,555.67"
1603,SIBRH,d37298df0abf3b83a000ed2f24f4f8bdd3004f9c,"-4,675,732.16",7cd93a932c3ce214bcc8dd9979a9f20f95e5c992,"-4,644,440.80",New,BCVA,"4,675,732.16","31,291.36","31,291.36"


In [4]:
dfmasked[dfmasked['Old/New'].isna()]

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
144,PTROL,50906ed51b7f535f56ae6ca7bc1abfcb2610e1cb,nan,53f1a38a6ca21ea5bd6ffd2b37133e9124983fa5,nan,NaN,NaN,nan,nan,nan
175,FSKSS,e6ca36cbcc53cb5dc4d962f2488d1ae90826e0bc,nan,d7aa32e782e1678ccb95b76ca487fcc0d8d9a99e,nan,NaN,NaN,nan,nan,nan
214,SNOVA,a6305a2357b631ffb73baf21b7310854600cb1b4,"-1,070.13",NaN,nan,NaN,NaN,"1,070.13",nan,nan
281,NRTXS,5216f73e72a094f81d0c52221331db2577b93c54,0.00,NaN,nan,NaN,NaN,0.00,nan,nan
287,MBDDR,d85095876d4db882e1e50d0be0a22e4b5504e578,nan,179df3e7601e030abc4cdcd33ef230011c861f16,nan,NaN,NaN,nan,nan,nan
342,NIZZZ,53146902254dec7d40d078e42845bfb7c90a7637,0.00,NaN,nan,NaN,NaN,0.00,nan,nan
502,AGOKK,f4c3ef2666ed296f1712b8184f8e07efc0a6f493,0.00,NaN,nan,NaN,NaN,0.00,nan,nan
633,GEKRY,661d9b14977d1d0265a0b51b62f19eb32ee7ff82,0.00,NaN,nan,NaN,NaN,0.00,nan,nan
670,ARNEV,5d728fcb9b9d2b5afc6ed207616f683b67d1392b,0.00,NaN,nan,NaN,NaN,0.00,nan,nan
695,URBEN,5e684b5df8aedced5381e564d168295a21df47d5,0.00,NaN,nan,NaN,NaN,0.00,nan,nan


In [21]:
6763/89

75.98876404494382

In [6]:
print("all=",dfmasked['BCVA_prev'].sum())
print("old=",dfmasked_old['BCVA_prev'].sum())
print("new=",dfmasked_new['BCVA_prev'].sum())
print("old+new=",dfmasked_old['BCVA_prev'].sum()+dfmasked_new['BCVA_prev'].sum())

all= -16287112.087532878
old= -6015092.59075744
new= -10272019.496775437
old+new= -16287112.087532878


In [22]:
dfmasked[dfmasked['Cpty']=='COMDK']

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
938,COMDK,6810db49425e08945925c1bace436a765bd6395b,-988.26,961ece9a820f33f0f424a892636d1c876485cd7d,"-8,989.82",Old,BCVA,988.26,"-8,001.56","8,001.56"


In [5]:


#reqv_tek=ps.get(resp_tek._nonce)
#reqv_prev=ps.get(resp_prev._nonce)
#big_request = prepare_big_request_for_pnl(reqv_tek, reqv_prev, algo_id,'USD', queries)

queries = ['BCVA','CVA','DVA']

big_request = prepare_big_request_for_pnl(ps.new_cm(resp_tek.Result), ps.new_cm(resp_prev.Result), algo_id,'USD', queries)

b = run_generic_revaluation_based_metric_change_explain_batch(big_request, algo_id, True,-1,True)

ps.put(b)
ps_utils.env.logger.info(f'Result id is {b._id}')

# df = pd.DataFrame(columns=['MetricType', 'Category', 'Counterparty', 'Value'])
# for result in a:
#     for metric in queries:
#         for category in result[metric]:
#             df = pd.concat([df, pd.DataFrame([{
#                 'MetricType': metric,
#                 'Category': category,
#                 'Counterparty': result.Request['_Counterparty'],
#                 'Value': result[metric][category]}])])

# df.to_excel('PnLExplain.xlsx', index=False)

[INFO] 2023-08-18 12:45:35,514 - Calculating Total/Expired/New and removing failed cpties
[INFO] 2023-08-18 12:47:59,700 - Requests collected. Request=b12c843ae9efc3ad3689e4113983ee1dd6e738ad. PrevRequest=05b5634118d55fd8b138e26b0a21d1f6bd68d112
[INFO] 2023-08-18 12:53:31,762 - Total calculated
[INFO] 2023-08-18 12:53:31,768 - Doing some preparations
[INFO] 2023-08-18 12:53:31,769 - Converting market data to canonical format
[INFO] 2023-08-18 12:53:32,144 - Converting done
[INFO] 2023-08-18 12:53:32,145 - Collecting dependencies
[INFO] 2023-08-18 12:57:43,122 - Requests collected. Request=da08c1348f9559a106dce9f7b91ad6eb49e68814. PrevRequest=080e3512ad8eb2a60c10edfcfc5fcad592c46756
[INFO] 2023-08-18 13:09:50,020 - Market dependencies collected
[INFO] 2023-08-18 13:09:50,037 - Preparations done
[INFO] 2023-08-18 13:09:50,038 - Calculating StaticDataSet change effect
[INFO] 2023-08-18 13:09:50,184 - StaticDataSet change not needed
[INFO] 2023-08-18 13:09:50,186 - Calculating Product chan

Exception: No connections to servers: ['fopricing-dev.msk.trd.ru:4004']

In [6]:
mas_cpties=[]
mas_pnl_explains=[]
mas_total_diff=[]
for el in b:
    mas_cpties.append(ps.get(el).Request['_Counterparty'])
    mas_pnl_explains.append(ps.get(el)._id)
    mas_total_diff.append(ps.get(el).BCVA['Total'])
    
df=pd.DataFrame({'Cpty':mas_cpties,'Explain_hash':mas_pnl_explains,'Total_diff': mas_total_diff})

newoldcpties=pd.read_excel('OldNewCpties.xlsx')
newoldcpties=newoldcpties[newoldcpties['Metric']=='BCVA']
dfotv=df.merge(newoldcpties,on='Cpty',how='left')
#dfotv.to_excel(str(datetime.datetime.now().date())+"PnLExplainEdinBase" + dttek+"--"+dtprev+".xlsx")
#dfotv.to_excel(writer, sheet_name='PnLExplainEdinBase', index=False)
dfotv.sort_values(by='Total_diff', ascending=False)

NameError: name 'b' is not defined

In [22]:
print("Total diff=",dfotv['Total_diff'].sum())
print("Old diff=",dfotv[dfotv['Old/New']=='Old']['Total_diff'].sum())
print("New diff=",dfotv[dfotv['Old/New']=='New']['Total_diff'].sum())

Total diff= 147792.65270916896
Old diff= 130529.00129433666
New diff= -143992.3979867108


In [33]:
dfotv[dfotv['Cpty']=='SSPEE']

,Cpty,Explain_hash,Total_diff,Old/New,Metric
1536,SSPEE,1890afa9b59c8535b0278479d29c8507b0cc70b2,"18,625.38",NaN,NaN


In [80]:
dfmasked[dfmasked['Cpty']=='EFIRN']

,Cpty,Resp_Hash_prev,BCVA_prev,Resp_Hash_tek,BCVA_tek,Old/New,Metric,abs_old_BCVA,diff_BCVA,abs_diff_BCVA
0,EFIRN,783992bc53aec0370c3ba07bb9111b6275d4bbd7,"-31,315.44",0ad206d81d58339a8bc2b856bb64d138a9e0e6a1,"-30,538.48",New,BCVA,"31,315.44",776.96,776.96


In [10]:
from collections import defaultdict
def populate_dict(resp_hash,tekcpty,otv):
    r=ps.get(resp_hash)
    for el in r['BCVA']:
        #print("el=",el)
        if (el=='Total'):
            otv['ActualDiff'][tekcpty]=r['BCVA'][el]
            otv['ActualDiff'].setdefault('Total',0)
            otv['ActualDiff']['Total']+=r['BCVA'][el]
        elif (el=='Unexplained'):
            otv['Unexplained'][tekcpty]=r['BCVA'][el]
            otv['Unexplained'].setdefault('Total',0)
            otv['Unexplained']['Total']+=r['BCVA'][el]
        elif (el=='Product'):
            otv['Product'][tekcpty]=(r['BCVA'][el],r._id)
            otv['Product'].setdefault('Total',0)
            otv['Product']['Total']+=r['BCVA'][el]
        elif (el=='Model.MarketDataSet.AsOfDate'):
            otv['Carry'][tekcpty]=r['BCVA'][el]
            otv['Carry'].setdefault('Total',0)
            otv['Carry']['Total']+=r['BCVA'][el]
            
        elif (el=='ErrorBoth'):
            otv['ErrorBoth'][tekcpty]=r['BCVA'][el]
            otv['ErrorBoth'].setdefault('Total',0)
            otv['ErrorBoth']['Total']+=r['BCVA'][el]
        elif (el=='ErrorCurrent'):
            otv['ErrorCurrent'][tekcpty]=r['BCVA'][el]
            otv['ErrorCurrent'].setdefault('Total',0)
            otv['ErrorCurrent']['Total']+=r['BCVA'][el]
        elif (el=='ErrorPrevious'):
            otv['ErrorPrevious'][tekcpty]=r['BCVA'][el]
            otv['ErrorPrevious'].setdefault('Total',0)
            otv['ErrorPrevious']['Total']+=r['BCVA'][el]
            
        elif (el=='ExpiredCounterparty'):
            otv['ExpiredCounterparty'][tekcpty]=r['BCVA'][el]
            otv['ExpiredCounterparty'].setdefault('Total',0)
            otv['ExpiredCounterparty']['Total']+=r['BCVA'][el]
        elif (el=='NewCounterparty'):
            otv['NewCounterparty'][tekcpty]=r['BCVA'][el]
            otv['NewCounterparty'].setdefault('Total',0)
            otv['NewCounterparty']['Total']+=r['BCVA'][el]
        elif (el=='Rating'):
            otv['Rating'][tekcpty]=r['BCVA'][el]
            otv['Rating'].setdefault('Total',0)
            otv['Rating']['Total']+=r['BCVA'][el]
        elif (el=='LGD'):
            otv['LGD'][tekcpty]=r['BCVA'][el]
            otv['LGD'].setdefault('Total',0)
            otv['LGD']['Total']+=r['BCVA'][el]
        elif (el=='NettingSetLGD'):
            otv['NettingSetLGD'][tekcpty]=r['BCVA'][el]
            otv['NettingSetLGD'].setdefault('Total',0)
            otv['NettingSetLGD']['Total']+=r['BCVA'][el]
        elif (el=='CreditModel'):
            otv['CreditModel'][tekcpty]=r['BCVA'][el]
            otv['CreditModel'].setdefault('Total',0)
            otv['CreditModel']['Total']+=r['BCVA'][el]
            
        elif (el.startswith('Model.MarketDataSet.CreditCurves')):
            curve=el.split('.')[3]
            if (curve not in otv['CreditCurves']):
                otv['CreditCurves'][curve]=defaultdict(defaultdict)
            otv['CreditCurves'][curve][tekcpty]=r['BCVA'][el]
            otv['CreditCurves'][curve].setdefault('Total',0)
            otv['CreditCurves'][curve]['Total']+=r['BCVA'][el]
            otv['CreditCurves'].setdefault('Total',0)
            otv['CreditCurves']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.GenericScalars')):
            otv['GenericScalars'][tekcpty]=r['BCVA'][el]
            otv['GenericScalars'].setdefault('Total',0)
            otv['GenericScalars']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.Spots')):
            pair=el.split('.')[3][0:6]
            if (pair not in otv['Spots']):
                otv['Spots'][pair]=defaultdict(defaultdict)
            otv['Spots'][pair][tekcpty]=r['BCVA'][el]
            otv['Spots'][pair].setdefault('Total',0)
            otv['Spots'][pair]['Total']+=r['BCVA'][el]
            otv['Spots'].setdefault('Total',0)
            otv['Spots']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.RatesCurvesBundles')):
            ccy=el.split('.')[3][0:3]
            curve=el.split('.')[3]
            if (ccy not in otv['Rates']):
                otv['Rates'][ccy]=defaultdict(defaultdict)
            otv['Rates'][ccy][curve][tekcpty]=r['BCVA'][el]
            otv['Rates'][ccy][curve].setdefault('Total',0)
            otv['Rates'][ccy][curve]['Total']+=r['BCVA'][el]
            otv['Rates'][ccy].setdefault('Total',0)
            otv['Rates'][ccy]['Total']+=r['BCVA'][el]
            otv['Rates'].setdefault('Total',0)
            otv['Rates']['Total']+=r['BCVA'][el]
        elif (el.startswith('Model.MarketDataSet.VolatilitySurfaces')):
            flagIsFXVol=True
            if (el.split('.')[3][3]=='_'):
                flagIsFXVol=False
            if not flagIsFXVol:
                ccy=el.split('.')[3][0:3]
                curve=el.split('.')[3]
                if (ccy not in otv['RatesVols']):
                    otv['RatesVols'][ccy]=defaultdict(defaultdict)
                otv['RatesVols'][ccy][curve][tekcpty]=r['BCVA'][el]
                otv['RatesVols'][ccy][curve].setdefault('Total',0)
                otv['RatesVols'][ccy][curve]['Total']+=r['BCVA'][el]
                otv['RatesVols'][ccy].setdefault('Total',0)
                otv['RatesVols'][ccy]['Total']+=r['BCVA'][el]
                otv['RatesVols'].setdefault('Total',0)
                otv['RatesVols']['Total']+=r['BCVA'][el]
            elif flagIsFXVol:
                pair=el.split('.')[3][0:6]
                if (pair not in otv['FXVols']):
                    otv['FXVols'][pair]=defaultdict(defaultdict)
                otv['FXVols'][pair][tekcpty]=r['BCVA'][el]
                otv['FXVols'][pair].setdefault('Total',0)
                otv['FXVols'][pair]['Total']+=r['BCVA'][el]
                otv['FXVols'].setdefault('Total',0)
                otv['FXVols']['Total']+=r['BCVA'][el]
        else:
            print("Do not know where to put into table el=",el, r['BCVA'][el])
                
                
            

    return otv

def get_pnl_table(a):
    bcva_vals=[]
    bcva_mas1=[]
    bcva_mas2=[]
    for el in a:
        if el in ['Carry','GenericScalars','ActualDiff','Unexplained','Product','Total',\
                  'ErrorBoth','ErrorPrevious','ErrorCurrent','ExpiredCounterparty','NewCounterparty',\
                  'Rating','LGD','NettingSetLGD','CreditModel']:
            bcva_mas1.append(el)
            bcva_mas2.append(el)
            bcva_vals.append(a[el]['Total'])
        elif el in ['CreditCurves','Rates','Spots','RatesVols','FXVols']:
            for k in a[el]:
                if k=='Total':
                    continue
                bcva_mas1.append(el)
                bcva_mas2.append(k)
                bcva_vals.append(a[el][k]['Total'])
        else:
            print("Bad el:",el)
    bcva_arrays = [np.array(bcva_mas1),np.array(bcva_mas2)]
    bcva_df=pd.DataFrame(bcva_vals, index=bcva_arrays)
    return bcva_df

In [12]:

dfotv_old=dfotv[dfotv['Old/New']=='Old'].reset_index(drop=True)

otv_old=defaultdict(defaultdict)
for i in range(len(dfotv_old)):
    otv_old=populate_dict(dfotv_old['Explain_hash'][i],dfotv_old['Cpty'][i],otv_old)
for el in otv_old:
    #print(el,otv_old[el]['Total'])
    pass
    
    
OldForSS=get_pnl_table(otv_old)
print("Sum of really explained (wo unexplained)=", OldForSS.values.sum()-OldForSS.loc['ActualDiff'].values-OldForSS.loc['Unexplained'].values)
#print("Explained=",OldForSS.loc['Explained'].values)
#print("Actual diff=",NAME['diff_BCVA'].sum())
#t.to_excel(str(datetime.datetime.now().date())+"OldBookTable" + dttek+"--"+dtprev+".xlsx")
#t.to_excel(writer, sheet_name='OldForSS', index=False)
OldForSS

Sum of really explained (wo unexplained)= [[122881.88451101]]


0
ActualDiff      ActualDiff           137,909.74
Carry           Carry                100,122.25
CreditCurves    RUB_MINFIN_OFZ             4.42
                RUB_SBER_OFZ_SNR_CR       56.35
Rates           RUB                  -71,407.96
                USD                    2,555.42
                CNH                   -3,362.03
                XAU                       -0.69
                EUR                     -543.26
                CHF                      -53.04
Spots           USDRUB                -5,177.98
                CNHRUB               207,897.60
                XAURUB                  -669.67
                EURRUB                12,742.10
                CHFRUB                12,249.69
RatesVols       RUB                 -126,943.36
                USD                        0.00
                EUR                        0.00
Unexplained     Unexplained           15,027.85
Product         Product               -4,594.62
NewCounterparty NewCounterparty            6.67

In [13]:
otv_old['Product']

defaultdict(None,
            {'ELKOR': (-82.28492565568945,
              'c907de724171ec604ebca78f6d99d5c8865fa19a'),
             'Total': -4594.619410419414,
             'RTRTR': (0.0, '8a93b46ad28f972d197df68eaa37b961b4ce7c00'),
             'SERVK': (0.0, 'a88b7fe1431c97be1b5a593a5ba33de966702bf5'),
             'SRBNN': (0.0, 'a49770cfc8f3e89462210baf2480908dc10af81e'),
             'INTSL': (1.6240688954628963,
              '3ccad7098dbf42cc2d31f7b499e8d274eff77aff'),
             'CVETA': (-1.8834693241559535,
              '536f0be9e0fb028666a60e8038d89dde2f42f5d0'),
             'ULTMR': (0.0, 'e5eb458cc10e406d07fcd3978566104d85deb753'),
             'CHERR': (15.18702114894404,
              'a5930137cc98820b34c8db588e13e45ce4852836'),
             'MEDEY': (0.0, '10419c72d31ce61aebb9fea48ce569248f315ca6'),
             'SBAGG': (0.0, '6a35b3588ae1b8c8cbe58f39b9798c2b99d217af'),
             'GRRRG': (0.0, '550d500ceab431a737b4c2bb721670f2726b2f34'),
             'RUSAS':

In [14]:


dfotv_newAndNone=dfotv[dfotv['Old/New']!='Old'].reset_index(drop=True)


otv_newAndNone=defaultdict(defaultdict)
for i in range(len(dfotv_newAndNone)):
    otv_newAndNone=populate_dict(dfotv_newAndNone['Explain_hash'][i],dfotv_newAndNone['Cpty'][i],otv_newAndNone)
for el in otv_newAndNone:
    #print(el,otv_new[el]['Total'])
    pass
    
    
NewForSS=get_pnl_table(otv_newAndNone)
print("Sum of really explained (wo unexplained)=",NewForSS.values.sum()-NewForSS.loc['ActualDiff'].values-NewForSS.loc['Unexplained'].values)
#print("Explained=",NewForSS.loc['Explained'].values)
#print("Actual diff=",NAME['diff_BCVA'].sum())
#NewForSS.to_excel(str(datetime.datetime.now().date())+"NewBookTable" + dttek+"--"+dtprev+".xlsx")
NewForSS

Sum of really explained (wo unexplained)= [[-71227.04489465]]


0
ActualDiff          ActualDiff           -56,244.96
Carry               Carry                  4,920.57
CreditCurves        RUB_MINFIN_OFZ          -114.84
                    RUB_SBER_OFZ_SNR_CR      -59.34
                    RUB_CHMF_OFZ_SNR_CR        0.00
Rates               RUB                    6,993.90
                    USD                   -3,546.60
                    CNH                    2,372.84
                    XAU                      496.94
                    EUR                  -13,812.06
                    XPD                       -0.00
Spots               USDRUB               357,995.78
                    CNHRUB              -442,183.83
                    XAURUB                -8,641.18
                    EURRUB               177,618.09
                    XPDRUB                    -0.25
RatesVols           RUB                  -25,086.57
                    USD                        0.00
                    EUR                        0.00
Unexplained         Unexplained           14,982.09
NewCounterparty     NewCounterparty     -111,031.42
Product             Product              -17,149.09
ErrorBoth           ErrorBoth                  0.00
ExpiredCounterparty ExpiredCounterparty        0.00

In [15]:
otv_newAndNone['Product']

defaultdict(None,
            {'FOTON': (-37.24767123960714,
              '2937173981628d056a52af4504e38497767bbcff'),
             'Total': -17149.094436402986,
             'SBKRR': (0.0, 'cae39d31e0b600f05912093f862a5dc8ef73dbce'),
             'ZONDX': (-3.831816007988891,
              '425157bd5f9650ef3e2343d8434ea9702f9d690a'),
             'DOTTE': (-2202.289336677006,
              '770875e6ad97dffcef08cd5630a11ac62499261e'),
             'DVAMR': (0.0, '0a07e60c7d62d9dbe4445deee461951a9c9c3493'),
             'UELMD': (-3.152299323977174,
              '3a25f633664d0ac7f768a63367e94321098a2113'),
             'KARMA': (-325.9966759146168,
              'e2ae4f0a113852cf00f0b20014d648572cbab50b'),
             'JDKEN': (0.0, 'a5a1fff166b2892954d99e4d97782d2468848ed9'),
             'NRTXS': (-0.07400103488016573,
              'c109da0e389d43420693d7a98ca635126411f1f5'),
             'SZUUU': (510.3338285259897,
              '057c6db6d8ccd8a11e54a39e321648b4065c57ef'),
     

In [28]:
dfotv_old[dfotv_old['Cpty']=='VOENSK']

,Cpty,Explain_hash,Total_diff,Old/New,Metric
655,VOENSK,f783aa96df907e0c6576e80d3b5a8b8edb84080a,"-7,304.74",Old,BCVA


In [21]:
dfotv_newAndNone[dfotv_newAndNone['Cpty']=='FGRPP']

,Cpty,Explain_hash,Total_diff,Old/New,Metric
808,FGRPP,413322e436ec21522bc8a1a95138039925e0bb46,"-1,452.14",NaN,NaN


In [16]:


t_0 = resp_prev.Result[0].Request.Model.MarketDataSet.AsOfDate
t_1 = resp_tek.Result[0].Request.Model.MarketDataSet.AsOfDate


ps_utils.env.logger.info('Saving results to Excel')
df = pd.DataFrame(columns=['MetricType', 'Category', 'Counterparty', 'Value'])
for result in b:
    for metric in queries:
        for category in result[metric]:
            df = pd.concat([df, pd.DataFrame([{
                'MetricType': metric,
                'Category': category,
                'Counterparty': result['_Counterparty'],
                'Old/New': dfotv[dfotv['Cpty']==result['_Counterparty']]['Old/New'].values[0],
                'Value': result[metric][category]}])])

df_aggr = pd.DataFrame(df.groupby(['MetricType', 'Category'])['Value'].apply(sum)).pivot_table(index='Category', columns=['MetricType'], values='Value').fillna(0)

#writer = pd.ExcelWriter(f'PnL Explain_{t_0.strftime("%Y-%m-%d")}_{t_1.strftime("%Y-%m-%d")}.xlsx', engine='xlsxwriter')
#df.to_excel(writer, sheet_name='PnLExplain', index=False)
#df_aggr.to_excel(writer, sheet_name='PnLAggr', index=True)

# wb = writer.book
# ws1 = writer.sheets['PnLExplain']
# ws2 = writer.sheets['PnLAggr']
# nb_format = wb.add_format({'num_format': '#,##0'})
# ws1.set_column(0, 0, 15)
# ws1.set_column(1, 1, 60)
# ws1.set_column(2, 2, 15)
# ws1.set_column(3, 3, 10, nb_format)
# ws2.set_column(0, 0, 60)
# ws2.set_column(1, 3, 10, nb_format)

# #writer.close()
# ps_utils.env.logger.info('Results saved!')




[INFO] 2023-08-14 16:17:52,379 - Saving results to Excel


In [17]:
excel_name=str(datetime.datetime.now().date())+"PnLExplainEdinBase" + dttek+"--"+dtprev+".xlsx"
writer = pd.ExcelWriter(excel_name, engine='xlsxwriter')

dfotv.to_excel(writer, sheet_name='PnLExplainEdinBase', index=False)

OldForSS.to_excel(writer, sheet_name='OldForSS')
OldProduct=pd.DataFrame({'Cpty': otv_old['Product'].keys(),'Product': otv_old['Product'].values()})
OldProduct.to_excel(writer, sheet_name='OldProduct', index=False)

NewForSS.to_excel(writer, sheet_name='NewForSS')
NewProduct=pd.DataFrame({'Cpty': otv_newAndNone['Product'].keys(),'Product': otv_newAndNone['Product'].values()})
NewProduct.to_excel(writer, sheet_name='NewProduct', index=False)

df.to_excel(writer, sheet_name='PnLExplainTimur', index=False)
df_aggr.to_excel(writer, sheet_name='PnLAggrTimur', index=True)

wb = writer.book
ws1 = writer.sheets['PnLExplainTimur']
ws2 = writer.sheets['PnLAggrTimur']
nb_format = wb.add_format({'num_format': '#,##0'})
ws1.set_column(0, 0, 15)
ws1.set_column(1, 1, 60)
ws1.set_column(2, 2, 15)
ws1.set_column(3, 3, 10, nb_format)
ws2.set_column(0, 0, 60)
ws2.set_column(1, 3, 10, nb_format)

ps_utils.env.logger.info('Results saved!')

writer.close()

[INFO] 2023-08-14 16:28:56,168 - Results saved!


In [13]:
m=0
for el in dfotv_old['Explain_hash']:
    el=ps.get(el)
    try:
        m+=el['BCVA']['Rating']
    except:
        continue
m

0

In [14]:
m=0
for el in dfotv_new['Explain_hash']:
    el=ps.get(el)
    try:
        m+=el['BCVA']['Rating']
    except:
        continue
m

0

In [32]:
s1=set()
r=ps.get('f25c540dd3487e6c69d2e61c98ce85adb9aec62d')
for a in r.Product.NettingSets:
    for b in a.MarginSets:
        for el in b.Products:
            s1.add(el['_ContractID'])
s2=set()
r=ps.get('de0fc77cbb8b5fd9f8ad3fcfaba3b94469355427')
for a in r.Product.NettingSets:
    for b in a.MarginSets:
        for el in b.Products:
            s2.add(el['_ContractID'])
print(len(s1))
print(len(s2))
print("Expired", s1.difference(s2))
print("New", s2.difference(s1))

40
38
Expired {'43037418', '43037446', '43037429'}
New {'44462918'}


In [19]:
len(s2)

117

In [15]:
r=ps.get('d8b6cedc2e23fb45dc2b36d87f6d017597b8a3a7')
s24=set()
for el in r.Products:
    s24.add(el._id)
s24

{'0240be1267bf6c619b31b0e7fc79e45cc3ed7830',
 '0516487e5544bc7570aa926ef6968b85dde487c9',
 '07349d1f04a2c33bdbf2eb2640175e49d2911228',
 '156f5e315afa78c47c98204db98a324c71ae2fb2',
 '18b013066cabdbd445c3cff5368fbca20da5f2e1',
 '20b7a11ad789029764998bab6d8d6b78da7af904',
 '2980f50d239f442e5b4ee24f1f03e5952ad4354b',
 '2e2623401a289e8bc68fa3a81447b12af03baf60',
 '34a828867c96005f4528374ee834b38ba357f1cb',
 '3b82a6f8d6ce19274020b61743d2c8213f079525',
 '425c045bc195f5ef03dcba11f8224eeee1fb5e10',
 '459b95b264d9b7a8c8290459787568d863f2920c',
 '467dca8fb89960732d8d29e5259e0cac02f00f73',
 '4d5f1dfcba290b7b399112cb7c9131fb497d1890',
 '55a0871f0b7abc7397e6ad10c2958e31ec7eadc6',
 '617394d5aef9fe6aa0d541b3a9907c89a21353c8',
 '62e4496eadabf8b8dc3bdeaadf2a0a507d68cda6',
 '74acd86f342ce1d2308560dadaaff98ad9f57261',
 '7ffae50624407f6a65d26b42c4f4032dbd04ef5d',
 '808b558ec6d802d2169ae72c018e5b3092d6af35',
 '80e9a4252f440c1ebca4c14321e00e22ede4e710',
 '95669b81c7feff672042eaddb7de9957abc73909',
 'a4ae4923

In [16]:
r=ps.get('fa3463877df3030d8d153307ffcb8a1b4addb464')
s25=set()
for el in r.Products:
    s25.add(el._id)
s25

{'0240be1267bf6c619b31b0e7fc79e45cc3ed7830',
 '0516487e5544bc7570aa926ef6968b85dde487c9',
 '07349d1f04a2c33bdbf2eb2640175e49d2911228',
 '156f5e315afa78c47c98204db98a324c71ae2fb2',
 '18b013066cabdbd445c3cff5368fbca20da5f2e1',
 '20b7a11ad789029764998bab6d8d6b78da7af904',
 '2980f50d239f442e5b4ee24f1f03e5952ad4354b',
 '2e2623401a289e8bc68fa3a81447b12af03baf60',
 '34a828867c96005f4528374ee834b38ba357f1cb',
 '3530f40fcd5995ebd57b49ff90c4532ce06241a2',
 '3b82a6f8d6ce19274020b61743d2c8213f079525',
 '425c045bc195f5ef03dcba11f8224eeee1fb5e10',
 '459b95b264d9b7a8c8290459787568d863f2920c',
 '467dca8fb89960732d8d29e5259e0cac02f00f73',
 '4d5f1dfcba290b7b399112cb7c9131fb497d1890',
 '55a0871f0b7abc7397e6ad10c2958e31ec7eadc6',
 '582a79ba1073d029686a2c23a03c93f8ce7c6926',
 '617394d5aef9fe6aa0d541b3a9907c89a21353c8',
 '62e4496eadabf8b8dc3bdeaadf2a0a507d68cda6',
 '65178d151724fa1f4601aa9c3c08ef504d44e45b',
 '6b55e4173d64577588b88bb43ed96e7ca5e11004',
 '74acd86f342ce1d2308560dadaaff98ad9f57261',
 '7e5f46ef

In [17]:
s25.difference(s24)

{'3530f40fcd5995ebd57b49ff90c4532ce06241a2',
 '582a79ba1073d029686a2c23a03c93f8ce7c6926',
 '65178d151724fa1f4601aa9c3c08ef504d44e45b',
 '6b55e4173d64577588b88bb43ed96e7ca5e11004',
 '7e5f46ef84a374cb911a1cbc97e6c977d1f87f2a',
 '7f0fd02acb42d1043d31dd3d9ba4186d69e1f2fd'}

In [19]:
ps.put(ps.new_cm(s25.difference(s24)))

CMSG<e097484274049079093b58362f21a680370c8bf8>[582a79ba1073d029686a2c23a03c93f8ce7c6926,65178d151724fa1f4601aa9c3c08ef504d44e45b,7f0fd02acb42d1043d31dd3d9ba4186d69e1f2fd,3530f40fcd5995ebd57b49ff90c4532ce06241a2,6b55e4173d64577588b88bb43ed96e7ca5e11004,7e5f46ef84a374cb911a1cbc97e6c977d1f87f2a,]

In [21]:
list(s25.difference(s24))

['582a79ba1073d029686a2c23a03c93f8ce7c6926',
 '65178d151724fa1f4601aa9c3c08ef504d44e45b',
 '7f0fd02acb42d1043d31dd3d9ba4186d69e1f2fd',
 '3530f40fcd5995ebd57b49ff90c4532ce06241a2',
 '6b55e4173d64577588b88bb43ed96e7ca5e11004',
 '7e5f46ef84a374cb911a1cbc97e6c977d1f87f2a']

In [18]:
ps_utils.switch_to_psi()
#as=[]
r=ps.get('3bfb921f83e290e226b5e0989c7716f508c0d18f')
for el in r.Result:
    if(ps.get(el._nonce)['_Counterparty'] in ['NESTR','VOENSK']):
        print(ps.get(el._nonce)['_Counterparty'],el._id)
        #as.append(ps.get(el._nonce)._id)
#s.put(ps.new_cm(mas))

[INFO] 2023-07-21 08:48:08,010 - Nothing to do: current environment is already psi


NESTR 08398cc9a51478b6023331ad858dae42927e9cea
VOENSK 4f5bfedb201f35d037f80dfe0d62f1a41755b385


In [63]:
import ps, ps_utils
#_from = 'dev'; _to = 'prod_pvlss'
_from = 'prod_pvlss'; _to = 'dev'
_id = '1d402a2ed5dcf164201b745ebec8088fee761f21'

ps_utils.upload_message(_id, _from, _to)

ps_utils.switch_to_env(_to)
print(f"check result")
n, not_found = ps_utils.chk_msg_tree(ps.get(_id))
print(f"\ndone: {n}, not found: {not_found}\n")
if not_found:
    print(f"copy missing messages")
    for m_id in not_found:
        ps_utils.upload_message(m_id, _from, _to)
    print(f"check result")
    n, not_found = ps_utils.chk_msg_tree(ps.get(_id))
    print(f"\ndone: {n}, not found: {not_found}\n")


[INFO] 2023-07-25 12:41:04,858 - Uploaded message 1d402a2ed5dcf164201b745ebec8088fee761f21 from prod_pvlss to dev
[INFO] 2023-07-25 12:41:04,865 - ps client switched from prod_pvlss to dev


check result
Can not load message ccd30c2e1c84e0394600aea879ccf55df5593596 by reference from 1d402a2ed5dcf164201b745ebec8088fee761f21 -> d9589897fff55546b1d23b0dfbb0ad6897ec89f0
Can not load message 46c3051e8e10ac7ac93e2939c64e6f23f6185f7c by reference from 1d402a2ed5dcf164201b745ebec8088fee761f21 -> eed8ce4e49c737e7b4c48bbfa45ea8c4cfd80db2
Can not load message 46c3051e8e10ac7ac93e2939c64e6f23f6185f7c by reference from 1d402a2ed5dcf164201b745ebec8088fee761f21 -> eed8ce4e49c737e7b4c48bbfa45ea8c4cfd80db2 -> 93cfe48f93387d5f1973430f43003fe41d0a48c6 -> 6790dad6c1715e4a02db0e0f5d31a33e0182c18e -> d3b0a88d743953542169d98f1e5c92ea6551c008 -> 2773b6332a0ae74683020003accf085959f7403e
Can not load message 69a565b8e5e4da8f470d42188da7bfe01efd97ec by reference from 1d402a2ed5dcf164201b745ebec8088fee761f21
Can not load message ccd30c2e1c84e0394600aea879ccf55df5593596 by reference from 1d402a2ed5dcf164201b745ebec8088fee761f21 -> 51b58ac7bd8c102f0a0604fb4b037e4d24f9d787

done: 300792, not found: [ccd

[INFO] 2023-07-25 12:43:19,931 - Uploaded message ccd30c2e1c84e0394600aea879ccf55df5593596 from prod_pvlss to dev
[INFO] 2023-07-25 12:43:19,977 - Uploaded message 46c3051e8e10ac7ac93e2939c64e6f23f6185f7c from prod_pvlss to dev
[INFO] 2023-07-25 12:43:19,990 - Nothing to do, message 46c3051e8e10ac7ac93e2939c64e6f23f6185f7c is already in dev
[INFO] 2023-07-25 12:43:47,547 - Uploaded message 69a565b8e5e4da8f470d42188da7bfe01efd97ec from prod_pvlss to dev
[INFO] 2023-07-25 12:43:47,572 - Nothing to do, message ccd30c2e1c84e0394600aea879ccf55df5593596 is already in dev


check result

done: 300945, not found: []



In [3]:
ps_utils.switch_to_dev()

r=ps_utils.pricing.XVA(ps.get('70b029165056868856bcb3ca95127cc47c968871')[0])
#r.Model.MarketDataSet='f0fd133994799ffc6aeeeceb85f501b27d321afb'
#r.set_queries(["ProductValue","CVA","DVA","BCVA","CVA_ParallelInterestRatesVega","DVA_ParallelInterestRatesVega","CVA_ParallelForeignExchangeVega","DVA_ParallelForeignExchangeVega","CVA_ParallelCreditDelta","DVA_ParallelCreditDelta","CVA_ParallelRho","DVA_ParallelRho","CVA_CurrencyExposure","DVA_CurrencyExposure"])




#r.Model.TypeName=skew_model
# r.set_nb_paths(2**17)
# r.Model.AmericanMonteCarloNumberOfPaths=6000
#r.set_model_parameter('LocalVolatilityRangeNumberOfStdDevs', 6.0)
#r.set_model_parameter('LocalVolatilityNumberOfSmoothingPoints',17)

r.Model.TypeName='HybridCheyette1FactorMCLocalVolatilityModel'
r.Model.LocalVolatilityMinimum=1e-13
r.Model.CalibrationFrequencyInDays=30
r.Model.LocalVolNumberOfTimeStepsPerYear=250
r.Model.MonteCarloMaxStepInHours=24

#r.CalculationMethod='AnalyticWithAAD'


#r.Product.CounterpartyCreditRating=8

#r.NumberOfPaths=500000
#r.Model.AmericanMonteCarloNumberOfPaths=8000
#r.Queries.append('ExpectedPositiveExposure')
# r.Queries.append("ParallelVegaBump")
# r.Queries.append("RhoBumpUpDown")

#r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=97.7058197934
# r.Product.NettingSets[0].MarginSets[0].Products[0].Notionals=list(df['Notional'].values)






# r.Model.MarketDataSet.RatesCurvesBundles['RUB_KEY_RATE'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['EUR_XCCY'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['CNH_FX'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['RUB_SOFR'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['RUB_RUONIA_OIS'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['EUR_EURIBOR_6M'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['EUR_RUB'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_SOFR'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_LIBOR_3M'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['USD_XCCY'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['XAU_USD'].InterpolationMethod='Linear'
# r.Model.MarketDataSet.RatesCurvesBundles['CNH_SOFR'].InterpolationMethod='Linear'

#r.Model.MarketDataSet.VolatilitySurfaces['EUR_EURIBOR_6M']='2a4a735bfe0560d36ba070ecf93a1955e37d9905'

#r.Model.MarketDataSet.Fixings['USD_LIBOR_3M']='3dd57498d8d94ae1dd56fa89bb64cf6161146b1b'



# r.Product.pop('CounterpartyCreditRating')
# r.Product.CounterpartyCreditCurveIdentifier='RUB_CHMF_OFZ_SNR_CR'

new_credit_values = {'RUB_SBER_OFZ_SNR_CR': 0.6, 'RUB_MINFIN_OFZ': 1.30}
for cr_curve, cr_value in new_credit_values.items():
    r.Model.MarketDataSet.CreditCurves[cr_curve].RatesSchedule.Rates = [cr_value / 0.75] * \
                                                                       len(r.Model.MarketDataSet.CreditCurves[
                                                                               cr_curve].RatesSchedule.Rates)


# r.Model.NumeraireCurrency='CNH'
# r.set_correlation('CNH_FX','EURCNH',0.0)
# r.set_correlation('EURCNH','EUR_EURIBOR_6M',0.0)

# r.set_correlation('EURRUB','CNHRUB',0.9)


# r.Product.NettingSets[0].MarginSets[0].Products=[r.Product.NettingSets[0].MarginSets[0].Products[2]]
# r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums=list(1.909/1.87*np.array(r.Product.NettingSets[0].MarginSets[0].Products[0].RollingPremiums))

#r.set_correlation('USDRUB','CNHRUB',0.9)

#r.Product.NettingSets[0].MarginSets[0].Products[0].Leg1FixedRates=list((12.135 +10/100)/100*np.ones(116))

r.set_model_parameter('UseRateAveragingApproximation', False)

z=r.price(algo_id='e4fe902c3d2e3233571dc28142c37e87a9554d7b')
z

[INFO] 2023-10-03 14:38:43,562 - Nothing to do: current environment is already dev
[INFO] 2023-10-03 14:38:44,525 - Compute:12575b00f0b8d704d51ac21848cffc57a88f8ddf->0d1d9b9a6cb303643697d9cec40b6deb5c89d145


Dict([('BCVA', -147875.81525601342),
      ('CVA', -170595.45875990664),
      ('DVA', 22719.643503893232),
      ('ProductValue', 1817041.8086278602),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 18431.181452564593),
      ('CashToSettleToday', 0.0)])